In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.AddEquity("SPY")
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', axis=1).plot()

In [ ]:
from keras.utils.generic_utils import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json

In [ ]:
start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [ ]:
qb = QuantBook()
symbol = qb.AddCrypto("BTCUSD", Resolution.Daily).Symbol
history = qb.History(symbol, start, end).loc[symbol]
history.head()

In [ ]:
daily_pct_change = history[["open", "high", "low", "close", "volume"]].pct_change().dropna()
df = daily_pct_change
df.head()

In [ ]:
indexes = df[((df.volume == float("inf")))].index
for i in indexes:
    df.at[i, "volume"] = max(df.volume.drop(indexes))

In [ ]:
n_steps = 30 
features = []
labels = []
for i in range(len(df) - n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df["close"].iloc[i+n_steps] >= 0:
        label = 1
    else:
        label = 0
    labels.append(label)

In [ ]:
import numpy as np
features = np.array(features)
labels = np.array(labels)

In [ ]:
train_length = int(len(features)*0.7)
X_train = features[:train_length]
X_test = features[train_length:]
y_train = labels[:train_length]
y_test = labels[train_length:]

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
test_length = int(len(features)*0.3)
X_train = features[test_length:]
X_test = features[:test_length]
y_train = labels[test_length:]
y_test = labels[:test_length]

In [ ]:
sum(y_train)/len(y_train)

In [ ]:
model = Sequential([Dense(30, input_shape = X_train[0].shape, activation = 'relu'), 
Dense(20, activation = 'relu'),
 Flatten(), 
 Dense(1, activation = 'sigmoid')])

In [ ]:
model.compile(loss = 'binary_crossentropy', optimizer='adam', metrics=['accuracy', 'mse'])
model.fit(X_train, y_train, epochs=5)

In [ ]:
y_hat = model.predict(X_test)

In [ ]:
results = pd.DataFrame({'y':y_test.flatten(), 'y_hat': y_hat.flatten()})

In [ ]:
results.plot(title = "Model Performance: predicted vs actual", figsize = (17,7))

In [ ]:
pred_train = model.predict(X_train) 
scores = model.evaluate(X_train, y_train, verbose=0)
print("Accuracy on training data: {}% \n Error on test data: {}".format(scores[1], 1-scores[1]))

pred_test = model.predict(X_test)
scores2 = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy on training data: {}% \n Error on test data: {}".format(scores2[1], 1-scores2[1]))

In [ ]:
model_str = json.dumps(serialize_keras_object(model))

In [ ]:
model_key = 'bitcoin_price_predictor'

In [ ]:
qb.ObjectStore.Save(model_key, model_str)

In [ ]:
if qb.ObjectStore.ContainsKey(model_key):
    model_str = qb.ObjectStore.Read(model_key)
    config = json.loads(model_str)['config']
    model = Sequential.from_config(config)


In [ ]:
testDate = datetime.now()

In [ ]:
df = qb.History(symbol, testDate - timedelta(40), testDate).loc[symbol]
df_change = df[["open", "high", "low", "close", "volume"]].pct_change().dropna()
model_input = []

for index, row in df_change.tail(30).iterrows():
    model_input.append(np.array(row))
model_input = np.array([model_input])

In [ ]:
if model.predict(model_input)[0][0] == 0:
    print("down")
else:
    print("up")